In [ ]:
# %% Deep learning - Section 10.83
#    Code challenge 8: minibatch size in the wine dataset

#    1) Run a mini-batch size parametric experiment by setting the batch size to 2^n, n=1,3,5,7,9
#    2) Use the binarised wine quality feture and all 11 features to predict it
#    3) Optional: track time needed to train the model with each batch size, and bar-plot it

# This code pertains a deep learning course provided by Mike X. Cohen on Udemy:
#   > https://www.udemy.com/course/deeplearning_x
# The "base" code in this repository is adapted (with very minor modifications)
# from code developed by the course instructor (Mike X. Cohen), while the
# "exercises" and the "code challenges" contain more original solutions and
# creative input from my side. If you are interested in DL (and if you are
# reading this statement, chances are that you are), go check out the course, it
# is singularly good.


In [ ]:
# %% Libraries and modules
import numpy               as np
import matplotlib.pyplot   as plt
import torch
import torch.nn            as nn
import seaborn             as sns
import copy
import torch.nn.functional as F
import pandas              as pd
import scipy.stats         as stats
import time

from torch.utils.data                 import DataLoader,TensorDataset
from sklearn.model_selection          import train_test_split
from google.colab                     import files
from torchsummary                     import summary
from IPython                          import display
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')


In [ ]:
# %% Solution 1
#    Use a sequential model


In [ ]:
# %% Load and prepare data

# Load
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
data = pd.read_csv(url,sep=';')

# Remove some outliers (see lec. 82 for why)
data = data[data['total sulfur dioxide']<200]

# Z-score all the variables but quality
cols2zscore = data.keys()
cols2zscore = cols2zscore.drop('quality')

for col in cols2zscore:
    mean_val  = np.mean(data[col])
    std_val   = np.std(data[col])
    data[col] = (data[col] - mean_val) / std_val

# Binarise quality
data.loc[:,'boolean_quality'] = 0
data.loc[data['quality']>5, 'boolean_quality'] = 1
data.loc[data['quality']<6, 'boolean_quality'] = 0 # Implicit but here for clarity

# Convert from pandas dataframe to PyTorch tensor
data_t = torch.tensor( data[cols2zscore].values ).float()
labels = torch.tensor( data['boolean_quality'].values ).float()

print(f'Data shape: {data_t.shape}')
print(f'Labels shape: {labels.shape}')

# Labels need to be multidimentional for PyTorch, not an array, and need to be long integers too
labels = labels[:,None]
labels = labels.long()
print(f'Proper labels shape: {labels.shape}')


In [ ]:
# %% Split into train and test data

# Split with scikitlearn
train_data,test_data,train_labels,test_labels = train_test_split(data_t,labels,test_size=0.1)

# Convert into PyTorch datasets
train_data = TensorDataset(train_data,train_labels)
test_data  = TensorDataset(test_data,test_labels)

# Convert into DataLoader objects
# > the train_loader is moved inside the train_model() function to allow a parametric test of the batch size
test_loader  = DataLoader(test_data,batch_size=test_data.tensors[0].shape[0])


In [ ]:
# %% Function to generate the model

def gen_model():

    # Architecture
    ANN = nn.Sequential(
             nn.Linear(11,32),
             nn.ReLU(),
             nn.Linear(32,32),
             nn.ReLU(),
             nn.Linear(32,32),
             nn.ReLU(),
             nn.Linear(32,2))

    # Loss function
    loss_fun = nn.CrossEntropyLoss()

    # Optimizer
    optimizer = torch.optim.SGD(ANN.parameters(),lr=0.01)

    return ANN,loss_fun,optimizer


In [ ]:
# %% Function to train the model

# Parameters
num_epochs = 1000

def train_model(ANN,loss_fun,optimizer):

    # Initialise accuracies
    train_acc = []
    test_acc  = []
    losses    = []

    # Loop over epochs
    for epoch_i in range(num_epochs):

        batch_acc  = []
        batch_loss = []

        for X,y in train_loader:

            # Forward propagation and loss
            yHat = ANN(X)
            loss = loss_fun(yHat,y.squeeze())

            # Only now do backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Batch training accuracy
            preds = torch.argmax(yHat, axis=1)
            batch_acc.append(100 * torch.mean((preds == y.squeeze()).float()).item())
            batch_loss.append(loss.item())

        # Average accuracy from batch
        train_acc.append(np.mean(batch_acc))
        losses.append(np.mean(batch_loss))

        # Test accuracy
        X,y = next(iter(test_loader))
        pred_labels = torch.argmax(ANN(X),axis=1)
        test_acc.append(  100*torch.mean((pred_labels==y.squeeze()).float()).item() )

    # Function output
    return train_acc,test_acc,losses


In [ ]:
# %% Parametric experiment over mini-batches size (track time as well)

# Takes about 18 mins for batch size 2^n, n=1,3,5,7,9
arr            = np.arange(1,10)
batch_size_exp = arr[arr % 2==1]
#batch_size_exp = np.arange(9,10)

train_acc    = np.zeros((num_epochs,len(batch_size_exp)))
test_acc     = np.zeros((num_epochs,len(batch_size_exp)))
losses       = np.zeros((num_epochs,len(batch_size_exp)))
elapsed_time = np.zeros((len(batch_size_exp),1))

for i,exp_i in enumerate(batch_size_exp):

        start_time = time.time()

        batch_size   = int(2**exp_i)
        train_loader = DataLoader(train_data,batch_size=batch_size,shuffle=True,drop_last=True)

        ANN,loss_fun,optimizer = gen_model()
        train_acc[:,i],test_acc[:,i],losses[:,i] = train_model(ANN,loss_fun,optimizer)

        elapsed_time[i] = time.time() - start_time
        print(f"Batch size 2^{exp_i} = {batch_size} completed in {elapsed_time[i, 0]:.2f} seconds")


In [ ]:
# %% Functions for 1D smoothing filter

# Improved for edge effects - adaptive window
def smooth_adaptive(x,k):
    smoothed = np.zeros_like(x)
    half_k   = k // 2

    for i in range(len(x)):
        start       = max(0, i-half_k)
        end         = min(len(x), i+half_k + 1)
        smoothed[i] = np.mean(x[start:end])

    return smoothed


In [ ]:
# %% Plotting

# Interesting to see how the model does not go above a certain threshold; others
# architecture might, but one must also consider that humans are variable, often
# unpredictable, and inconsistent, thus if we develop models to predict human
# behaviour, it is silly to expect the model to have really high accuracy (e.g.,
# we migt like different wines, and we might like the same wine differently depending
# on the context)

fig,ax = plt.subplots(1,2,figsize=(17,7))

cmaps = plt.cm.plasma(np.linspace(.1,.9,len(batch_size_exp)))
for i in range(len(batch_size_exp)):
    ax[0].plot(smooth_adaptive(train_acc[:,i],20),color=cmaps[i])
    ax[1].plot(smooth_adaptive(test_acc[:,i],20),color=cmaps[i])

ax[0].set_title('Train accuracy')
ax[1].set_title('Test accuracy')

# Make the legend easier to read
leglabels = [2**int(i) for i in batch_size_exp]

# Common features
for i in range(2):
    ax[i].legend(leglabels)
    ax[i].set_xlabel('Epoch')
    ax[i].set_ylabel('Accuracy (%)')
    ax[i].set_ylim([30,101])
    ax[i].grid()

plt.savefig('figure8_code_challenge_8.png')

plt.show()

files.download('figure8_code_challenge_8.png')


In [ ]:
# %% Plotting

phi = ( 1 + np.sqrt(5) ) / 2
fig = plt.figure(figsize=(6*phi,6))

labels = [f"{2**i}" for i in batch_size_exp]
plt.bar(labels,elapsed_time.squeeze())
plt.xlabel('Batch Size')
plt.ylabel('Time (s)')
plt.title('Training time per minibatch size')

plt.savefig('figure9_code_challenge_8.png')

plt.show()

files.download('figure9_code_challenge_8.png')


In [ ]:
# %% Solution 2
#    Use a class


In [ ]:
# %% Load and prepare data

# Load
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
data = pd.read_csv(url,sep=';')

# Remove some outliers (see lec. 82 for why)
data = data[data['total sulfur dioxide']<200]

# Z-score all the variables but quality
cols2zscore = data.keys()
cols2zscore = cols2zscore.drop('quality')

for col in cols2zscore:
    mean_val  = np.mean(data[col])
    std_val   = np.std(data[col])
    data[col] = (data[col] - mean_val) / std_val

# Binarise quality
data.loc[:,'boolean_quality'] = 0
data.loc[data['quality']>5, 'boolean_quality'] = 1
data.loc[data['quality']<6, 'boolean_quality'] = 0 # Implicit but here for clarity

# Convert from pandas dataframe to PyTorch tensor
data_t = torch.tensor( data[cols2zscore].values ).float()
labels = torch.tensor( data['boolean_quality'].values ).float()

print(f'Data shape: {data_t.shape}')
print(f'Labels shape: {labels.shape}')

# Labels need to be multidimentional for PyTorch, not an array, and need to be long integers too
labels = labels[:,None]
print(f'Proper labels shape: {labels.shape}')


In [ ]:
# %% Split into train and test data

# Split with scikitlearn
train_data,test_data,train_labels,test_labels = train_test_split(data_t,labels,test_size=0.1)

# Convert into PyTorch datasets
train_data = TensorDataset(train_data,train_labels)
test_data  = TensorDataset(test_data,test_labels)

# Convert into DataLoader objects
# > the train_loader is moved inside the train_model() function to allow a parametric test of the batch size
test_loader  = DataLoader(test_data,batch_size=test_data.tensors[0].shape[0])


In [ ]:
# %% Model class

class model_class(nn.Module):
    def __init__(self):
        super().__init__()

        # Layers
        self.input  = nn.Linear(11,32)
        self.hid1   = nn.Linear(32,32)
        self.hid2   = nn.Linear(32,32)
        self.output = nn.Linear(32,1)

    # Forward propagation
    def forward(self,x):

        x = F.relu(self.input(x))
        x = F.relu(self.hid1(x))
        x = F.relu(self.hid2(x))
        x = self.output(x)
        return x


In [ ]:
# %% Function to train the model

# Parameters
num_epochs = 1000

def train_model():

    # Loss function and optimizer
    loss_fun = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.SGD(ANN.parameters(),lr=0.01)

    # Initialise losses
    train_acc = []
    test_acc  = []
    losses    = []

    # Loop over epochs
    for epoch_i in range(num_epochs):

        # Switch training mode on
        ANN.train()

        batch_acc  = []
        batch_loss = []

        for X,y in train_loader:

            # Forward propagation and loss
            yHat = ANN(X)
            loss = loss_fun(yHat,y)

            # Only now do backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Batch training accuracy
            batch_acc.append( 100*torch.mean(((yHat>0) == y).float()).item() )
            batch_loss.append(loss.item())

        # Average accuracy from batch
        train_acc.append(np.mean(batch_acc))
        losses.append(np.mean(batch_loss))

        # Test accuracy
        ANN.eval()
        X,y = next(iter(test_loader))
        with torch.no_grad():
            yHat = ANN(X)
        test_acc.append( 100*torch.mean(((yHat>0) == y).float()).item() )

    # Function output
    return train_acc,test_acc,losses


In [ ]:
# %% Parametric experiment over mini-batches size (track time as well)

# Takes about 17 mins for batch size 2^n, n=1,3,5,7,9
arr            = np.arange(1,10)
batch_size_exp = arr[arr % 2==1]

train_acc    = np.zeros((num_epochs,len(batch_size_exp)))
test_acc     = np.zeros((num_epochs,len(batch_size_exp)))
losses       = np.zeros((num_epochs,len(batch_size_exp)))
elapsed_time = np.zeros((len(batch_size_exp),1))

for i,exp_i in enumerate(batch_size_exp):

        start_time = time.time()

        batch_size   = int(2**exp_i)
        train_loader = DataLoader(train_data,batch_size=batch_size,shuffle=True,drop_last=True)

        ANN = model_class()
        train_acc[:,i],test_acc[:,i],losses[:,i] = train_model()

        elapsed_time[i] = time.time() - start_time
        print(f"Batch size 2^{exp_i} = {batch_size} completed in {elapsed_time[i, 0]:.2f} seconds")


In [ ]:
# %% Plotting

fig,ax = plt.subplots(1,2,figsize=(17,7))

cmaps = plt.cm.plasma(np.linspace(.1,.9,len(batch_size_exp)))
for i in range(len(batch_size_exp)):
    ax[0].plot(smooth_adaptive(train_acc[:,i],20),color=cmaps[i])
    ax[1].plot(smooth_adaptive(test_acc[:,i],20),color=cmaps[i])

ax[0].set_title('Train accuracy')
ax[1].set_title('Test accuracy')

# Make the legend easier to read
leglabels = [2**int(i) for i in batch_size_exp]

# Common features
for i in range(2):
    ax[i].legend(leglabels)
    ax[i].set_xlabel('Epoch')
    ax[i].set_ylabel('Accuracy (%)')
    ax[i].set_ylim([41,101])
    ax[i].grid()

plt.savefig('figure10_code_challenge_8.png')

plt.show()

files.download('figure10_code_challenge_8.png')


In [ ]:
# %% Plotting

phi = ( 1 + np.sqrt(5) ) / 2
fig = plt.figure(figsize=(6*phi,6))

labels = [f"{2**i}" for i in batch_size_exp]
plt.bar(labels,elapsed_time.squeeze())
plt.xlabel('Batch Size')
plt.ylabel('Time (s)')
plt.title('Training time per minibatch size')

plt.savefig('figure11_code_challenge_8.png')

plt.show()

files.download('figure11_code_challenge_8.png')


In [ ]:
# %% Exercise 1
#    There is another regularization technique called "early stopping," which simply means to stop training the model
#    earlier than the number of epochs you specified. Early stopping is used when the test accuracy starts to decrease
#    with increased training. Do you think that early stopping would be beneficial here? How many epochs would you train?

# In thise case the test accuracy reaches a plateau after a while, which is why an early
# stopping might not be an optimal choice; also, if implemented, there should be some tolerance
# to avoid small random fluctuations from stopping the training (consider that the plot are smoothed)


In [ ]:
# %% Exercise 2
#    The training loop computes the losses, but those aren't plotted. Create an additional subplot to show the losses
#    in a similar fashion as the accuracy. Does that plot provide any additional insights into the effects of minibatch
#    size, beyond what we can already learn from the accuracy plots.

# As one would expect from the training accuracy, the losses are higher for models
# with larger batches sizes; still interesting, however, that the test accuracy reaches
# approximately the same level despite the differences in training accuracy and losses

# Plotting
phi = ( 1 + np.sqrt(5) ) / 2
fig,ax = plt.subplots(1,3,figsize=(7*phi*1.5,7))

cmaps = plt.cm.plasma(np.linspace(.1,.9,len(batch_size_exp)))
for i in range(len(batch_size_exp)):
    ax[0].plot(smooth_adaptive(train_acc[:,i],20),color=cmaps[i])
    ax[1].plot(smooth_adaptive(test_acc[:,i],20),color=cmaps[i])
    ax[2].plot(smooth_adaptive(losses[:,i],20),color=cmaps[i])

ax[0].set_title('Train accuracy')
ax[1].set_title('Test accuracy')
ax[2].set_title('Losses')

ax[0].set_ylim([41,101])
ax[1].set_ylim([41,101])

# Make the legend easier to read
leglabels = [2**int(i) for i in batch_size_exp]

# Common features
for i in range(3):
    ax[i].legend(leglabels)
    ax[i].set_xlabel('Epoch')
    ax[i].set_ylabel('Accuracy (%)')
    ax[i].grid()

plt.savefig('figure12_code_challenge_8_extra2.png')

plt.show()

files.download('figure12_code_challenge_8_extra2.png')
